In [ ]:
import sys
sys.path.append('../..')
import pandas as pd
import numpy as np
import src.cleaning as clean
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
sales_data = pd.read_csv('../../data/anon_data.csv')
cleaned_data = clean.clean_data(sales_data)

In [ ]:
pivotted_brand_product = cleaned_data.pivot_table(values='quantity', index=['customer_id'], columns=['brand', 'product_type']).fillna(0)

In [ ]:
pivotted_brand_product.columns.shape
# This will likely not be applicable as there are too many feature, each with extremely high and low values, scaling these won't help enough as high dimensionality leads to high distance
# However will have a look anyway

In [ ]:
results = []
k_range = list(range(20,80))
for k in k_range:
    model = KMeans(n_clusters=k, random_state=26)
    model.fit(pivotted_brand_product)
    results.append(model.inertia_)


In [ ]:
plt.plot(k_range, results)
plt.grid(True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler(feature_range=(0,10))
new_data = scalar.fit_transform(pivotted_brand_product)

In [ ]:
np.unique(new_data, return_counts=True)

In [ ]:
results = []
k_range = list(range(20,80))
for k in k_range:
    model = KMeans(n_clusters=k, random_state=26)
    model.fit(new_data)
    results.append(model.inertia_)

In [ ]:
plt.plot(k_range, results)
plt.grid(True)

Not likely to be any good as inertia is still high even after scaling. Look to scale on different features

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
clusterer = KMeans(n_clusters=80, random_state=26)
clusters = clusterer.fit_predict(new_data)

In [ ]:
len(model.labels_)

In [ ]:
silhouette_score(new_data, clusterer.labels_)

In [ ]:
clusterer.cluster_centers_

# Consider PCA

In [ ]:
from sklearn.decomposition import PCA


In [ ]:
pca = PCA(0.95)

In [ ]:
pca_data = pca.fit_transform(new_data)

In [ ]:
pca.n_components_

Looking for 95% explained variance, PCA still gives 554 dimensions which is a lot

In [ ]:
pca_data.shape

In [ ]:
# Use elbow method to find optimal k
results = []
silhouettes = []
k_range = list(range(2, 40))
for k in k_range:
    k_mean_clusterer = KMeans(n_clusters=k, random_state=26)
    k_mean_clusterer.fit(pca_data)
    silhouettes.append(silhouette_score(pca_data, k_mean_clusterer.labels_))
    results.append(k_mean_clusterer.inertia_)

In [ ]:
plt.plot(k_range, results)
plt.grid(True)
# K c


In [ ]:
plt.plot(k_range, silhouettes)
plt.grid(True)
# K c